In [72]:
!pip3 install git+https://github.com/holounic/Bayesian-Optimization.git
!pip3 install cma

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/holounic/Bayesian-Optimization.git to /tmp/pip-req-build-j1ioaofm
  Running command git clone -q https://github.com/holounic/Bayesian-Optimization.git /tmp/pip-req-build-j1ioaofm
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Cloning https://github.com/holounic/pyDOE to /tmp/pip-install-wvd82x2c/pydoe_82572c6135d44b069f00cafd6796557a
  Running command git clone -q https://github.com/holounic/pyDOE /tmp/pip-install-wvd82x2c/pydoe_82572c6135d44b069f00cafd6796557a
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [73]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

import cma

from pyDOE import lhs

from bayes_optim import BO
from bayes_optim.search_space import RealSpace
from bayes_optim.surrogate import GaussianProcess, trend

from benchmark.bbobbenchmarks import F9

from tqdm.notebook import trange

# Constants

In [74]:
np.random.seed(1111)

dim = 2
lb, ub = -5, 5

mean = trend.constant_trend(dim, beta=None)
thetaL = 1e-10 * (ub - lb) * np.ones(dim)
thetaU = 10 * (ub - lb) * np.ones(dim)
theta0 = np.random.rand(dim) * (thetaU - thetaL) + thetaL

max_FEs = 2**5
verbose = False

doe_budget = 0.2
doe_size = int(doe_budget * max_FEs)

n_samples = 30000

max_cma_iter = 20

f = F9

test_seeds = list(range(50))

space = RealSpace([lb, ub]) * dim

pop_size = int(4 + 3 * np.log(doe_size * dim) // 1)

n_tests = 8

f_name = 'f9'

# Make optimizer function

In [77]:
def make_optimizer(obj_func, doe):
    model = GaussianProcess(
        mean=mean,
        corr="squared_exponential",
        theta0=theta0,
        thetaL=thetaL,
        thetaU=thetaU,
        nugget=0,
        noise_estim=False,
        optimizer="BFGS",
        wait_iter=3,
        random_start=dim,
        likelihood="concentrated",
        eval_budget=100 * dim
    )
    return BO(
        search_space=space,
        obj_fun=obj_func,
        model=model,
        DoE_size=len(doe),
        acquisition_fun="EI",
        initial_points=doe,
        max_FEs=max_FEs,
        verbose=verbose,
        n_point=1
    )

In [78]:
samples = space.sample(100000)

# Experiment code

In [79]:
def normalized_regret(obj_func, doe):
  bo = make_optimizer(obj_func, doe)
  xopt, yopt, _ = bo.run()
  return (yopt - obj_func.getfopt()) / (obj_func(samples).max() - obj_func.getfopt())

def regret(obj_func, doe):
  bo = make_optimizer(obj_func, doe)
  xopt, yopt, _ = bo.run()
  return yopt - obj_func.getfopt()

def fitness_function(doe_size, n_individuals, func_seeds):
  i = 0
  def func(X):
    nonlocal i
    X = X.clip(lb, ub, out=X)
    doe = np.split(X, doe_size)
    obj_func = None
    k = i // n_individuals
    func_seed_set = func_seeds[k]
    i += 1
    r = 0
    print(func_seed_set)
    for func_seed in func_seed_set:
      r += normalized_regret(f(func_seed), doe)
    return r / len(func_seed_set)
  return func

def cma_es_experiment(doe_size, func_seeds, individual0, sigma0 = ub / 3):
  # run cma-es 
  doe_flattened, es = cma.fmin2(fitness_function(doe_size, pop_size, func_seeds), individual0, sigma0, {'maxiter': max_cma_iter, 'popsize': pop_size})
  
  # split a vector into set of points
  doe = np.split(doe_flattened, doe_size)

  return doe, es

# Testing code

In [80]:
def additional_metrics(history):
  for h in history:
    func = f(h['f_s'])
    y = func(h['xopt'])
    real_x = func._getxopt()
    d = np.linalg.norm(h['doe'] - real_x, axis=1)
    h['min_dist_to_opt'] = d.min()
    h['max_dist_to_opt'] = d.max()
    h['real_xopt'] = real_x
    h['real_yopt'] = func.getfopt()
    h['bo_history'] = h['bo'].history
  return history

In [81]:
def compute_all(func_seed, doe, n=3, n_points=1000):
  regrets, history, normalized_regret = [], [], []
  for _ in range(n):
    obj_func = f(func_seed)
    bo = make_optimizer(obj_func, doe)
    xopt, yopt, _ = bo.run()
    regret = yopt - obj_func.getfopt()
    n_regret = (yopt - obj_func.getfopt()) / (obj_func(samples).max() - obj_func.getfopt())
    normalized_regret.append(n_regret)
    regrets.append(regret)
    history.append({'xopt': xopt, 'yopt': yopt, 'regret': regret, 'doe': doe, 'normalized_regret': n_regret, 'bo': bo, 'f_s': func_seed})
  return np.concatenate(regrets).mean(), np.mean(normalized_regret), history

def test(doe, n=10):
  n_regrets, regrets, history = [], [], []
  for seed in trange(n):
    doe = doe
    r, p, h = compute_all(seed, doe, n=1, n_points=1000)
    n_regrets.append(p)
    regrets.append(r)
    history.append(h[0])
  return n_regrets, regrets, history

# Experiments

In [82]:
individual0 = np.concatenate(lhs(2, doe_size) * 10 - 5)
individual0

array([ 3.81537132, -2.25215555,  0.67872627,  3.86894295,  2.61014304,
        0.37130287, -2.87200333, -1.47553649, -1.38993684,  3.22258113,
       -4.67863124, -3.8625861 ])

In [83]:
func_seeds = np.random.randint(0, 2**31, (max_cma_iter, n_tests)).tolist()
func_seeds.append(test_seeds)
func_seeds

[[1475636620,
  1134109346,
  1333488308,
  1005703320,
  8632214,
  651166484,
  1011871627,
  586793230],
 [1021307656,
  1790440934,
  1013250956,
  23639534,
  2128019186,
  1941953366,
  1221597576,
  441380969],
 [543340478,
  721493309,
  458055347,
  1529544234,
  2001974348,
  985630174,
  1018474702,
  2025616588],
 [1868994436,
  1247560269,
  1046561192,
  104666121,
  1648579849,
  1890662871,
  1453389842,
  1597818624],
 [666513316,
  104385800,
  637573627,
  705513029,
  597623957,
  101841489,
  590019245,
  1874809778],
 [476201470,
  1524655072,
  1846915629,
  319501579,
  1359356797,
  114995871,
  1157171255,
  1425728285],
 [366794197,
  1281455660,
  708681325,
  1428655094,
  909226738,
  2134097890,
  1663587672,
  1748947072],
 [1151088343,
  1358540125,
  1044006575,
  1473510937,
  1242557632,
  1876094952,
  1623430708,
  9520565],
 [1264779723,
  1077661487,
  1577674721,
  2113248958,
  1270514153,
  1999468482,
  1791778377,
  1561099687],
 [1164463400

In [84]:
doe, es = cma_es_experiment(doe_size, func_seeds, individual0, sigma0 = ub / 3)

(5_w,11)-aCMA-ES (mu_w=3.4,w_1=42%) in dimension 12 (seed=813284, Thu Nov 17 17:51:39 2022)
[1475636620, 1134109346, 1333488308, 1005703320, 8632214, 651166484, 1011871627, 586793230]
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
[1475636620, 1134109346, 1333488308, 1005703320, 8632214, 651166484, 1011871627, 586793230]
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
[1475636620, 1134109346, 1333488308, 1005703320, 8632214, 651166484, 1011871627, 586793230]
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
[1475636620, 1134109346, 1333488308, 1005703320, 8632214, 651166484, 1011871627, 586793230]
Invalid likelihood valu

In [85]:
doe

[array([ 2.86783222, -0.87896475]),
 array([3.50583436, 2.15926442]),
 array([ 0.62208433, -1.97116941]),
 array([0.83372785, 4.30807374]),
 array([-0.21015405,  0.31129987]),
 array([-4.33954173,  0.59992901])]

In [86]:
n_regrets, regrets, history = test(doe, n=50)

  0%|          | 0/50 [00:00<?, ?it/s]

Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likel

In [87]:
np.mean(n_regrets)

2.0970928943760386e-05

In [88]:
np.mean(regrets)

4.637383735579166

In [89]:
pd.DataFrame(additional_metrics(history)).drop('bo', axis=1).to_csv(f'{f_name}_cma_es_doe_normalized.csv')

In [90]:
doe_lhs = np.split(individual0, doe_size)
doe_lhs

[array([ 3.81537132, -2.25215555]),
 array([0.67872627, 3.86894295]),
 array([2.61014304, 0.37130287]),
 array([-2.87200333, -1.47553649]),
 array([-1.38993684,  3.22258113]),
 array([-4.67863124, -3.8625861 ])]

In [91]:
lhs_n_regrets, lhs_regrets, lhs_history = test(doe_lhs, n=50)

  0%|          | 0/50 [00:00<?, ?it/s]

Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...
Invalid likelihood value. Increasing nugget...


In [92]:
np.mean(lhs_n_regrets)

2.0476225395022323e-05

In [93]:
np.mean(lhs_regrets)

4.045129889471116

In [94]:
pd.DataFrame(additional_metrics(lhs_history)).drop('bo', axis=1).to_csv(f'{f_name}_lhs_initial_doe_normalized.csv')